In [ ]:
# Ponemos como ruta principal la carpeta padre de "soluciones"
# para que las rutas relativas queden igual que en la carpeta de notebooks
import paths

# Otras dependencias
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

#### SOS: ¡me han baneado!

Opción de usar el html offline que hay en `dat` para seguir con el ejercicio. Descomenta el siguiente código y sigue adelante

In [ ]:
# Descomenta esto si te han baneado
# with open('dat/milanuncios.html') as f:
#     soup = BeautifulSoup(f, 'html.parser')

### Ejercicio

Crea un dataframe de pandas en el que cada fila sea un anuncio y tenga como columnas información que consideres relevante: precio, kilómetros, año, cilindrada, texto del anuncio, ...

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}

page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

columns = ['producto','tipo_anuncio','precio','texto','cc','ano','kms']

df = pd.DataFrame(columns=columns)

for link in soup.find_all('div', class_='aditem ParticularCardTestABClass'):
    producto = link.find('a', class_='aditem-detail-title').get_text()
    tipo_anuncio = link.find('div', class_='x3').get_text()
    if link.find('div', class_='aditem-price') is None:
        precio = 0
    else:
        precio = link.find('div', class_='aditem-price').get_text().replace('€','')
    texto = link.find('div', class_='tx').get_text()
    if link.find('div', class_='cc tag-mobile') is None:
        cc = 0
    else:
        cc = link.find('div', class_='cc tag-mobile').get_text().replace('cc','')
    if link.find('div', class_='ano tag-mobile') is None:
        ano = 0
    else:
        ano = link.find('div', class_='ano tag-mobile').get_text().replace('año','') #¿Alguna solución mejor que un if para esto?
    if link.find('div', class_='kms tag-mobile') is None:
        kms = 0
    else:
        kms = link.find('div', class_='kms tag-mobile').get_text().replace('kms','')

    nueva_fila = {'producto':producto,'tipo_anuncio':tipo_anuncio,'precio':precio,'texto':texto,'cc':cc,'ano':ano,'kms':kms}
    df = df.append(nueva_fila, ignore_index=True)
    
df

### Ejercicio

Modifica el código anterior para que, además de bajarse la página actual, navegue por el resto de páginas e incorpore también esos anuncios a tu dataframe.

In [ ]:
page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

columns = ['producto','tipo_anuncio','precio','texto','cc','ano','kms']
df = pd.DataFrame(columns=columns)
nueva_pag = 1
while len(soup.find_all('div', class_='aditem ParticularCardTestABClass')) > 0:
    for link in soup.find_all('div', class_='aditem ParticularCardTestABClass'):
        producto = link.find('a', class_='aditem-detail-title').get_text()
        tipo_anuncio = link.find('div', class_='x3').get_text()
        if link.find('div', class_='aditem-price') is None:
            precio = 0
        else:
            precio = link.find('div', class_='aditem-price').get_text().replace('€','')
        texto = link.find('div', class_='tx').get_text()
        if link.find('div', class_='cc tag-mobile') is None:
            cc = 0
        else:
            cc = link.find('div', class_='cc tag-mobile').get_text().replace('cc','')
        if link.find('div', class_='ano tag-mobile') is None:
            ano = 0
        else:
            ano = link.find('div', class_='ano tag-mobile').get_text().replace('año','') #¿Alguna solución mejor que un if para esto?
        if link.find('div', class_='kms tag-mobile') is None:
            kms = 0
        else:
            kms = link.find('div', class_='kms tag-mobile').get_text().replace('kms','')

        nueva_fila = {'producto':producto,'tipo_anuncio':tipo_anuncio,'precio':precio,'texto':texto,'cc':cc,'ano':ano,'kms':kms}
        df = df.append(nueva_fila, ignore_index=True)

    nueva_pag += 1
    page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm?pagina='+str(nueva_pag), headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')

df

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [ ]:
# Pasos previos al ejercicio que se ejecutan durante el notebook de explicación
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content
soup = BeautifulSoup(contenido, "html.parser")
tablas = soup.find_all('table')
lineas = [x for tabla in tablas for x in tabla.find_all('tr')]
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas]
datos = [x for x in datos if len(x) > 0]
datos = pd.DataFrame(datos)
datos

#### Ejercicio

Usa los elementos `th` de la primera fila de las tablas para extraer nombres para las columnas de la tabla. 

In [ ]:
cabecera = [[x.text for x  in linea.find_all('th')] for linea in lineas]
cabecera = [x for x in cabecera if len(x) > 0]
cabecera = cabecera[0]
cabecera

#### Ejercicio

Elimina las columnas irrelevantes y cambia los nombres de las columnas por otros breves y sin caracteres extraños o que dificulten el posproceso.

In [ ]:
cabecera = [[x.text for x  in linea.find_all('th')] for linea in lineas]
cabecera = [x for x in cabecera if len(x) > 0]
cabecera = cabecera[0]
cabecera.pop(2) #Elimino la columna que tiene el nombre en blanco

cabecera = [re.sub('[()/. ]','',x) for x in cabecera]
cabecera = [re.sub('ó','o',x) for x in cabecera]
cabecera = [re.sub('%','pct',x) for x in cabecera]
cabecera = [re.sub('€','Euros',x) for x in cabecera]
cabecera[8] = 'Fecha'

datos = datos.drop([2], axis = 1) #Elimino una columna en blanco

datos.columns = cabecera

datos

#### Ejercicio

Cambia el formato de las columnas adecuadamente: convierte a numéricas, etc., las columnas que lo requieran.

In [ ]:
datos['VolumenEuros'] = datos['VolumenEuros'].replace(regex=r'\.',value='')

datos = datos.replace(regex=r'%',value='')
datos = datos.replace(regex=r',',value='.')
datos[['Precio','Varpct','VarEuros','VolumenEuros','Capitalizacion1','PER','RentDiv']] = datos[['Precio','Varpct','VarEuros','VolumenEuros','Capitalizacion1','PER','RentDiv']].astype(float)

datos['Fecha'] = datos['Fecha'] + '/2020'

datos['Fecha'] = pd.to_datetime(datos['Fecha'], format='%d/%m/%Y')

datos.dtypes